##### 포즈 landmarks 좌표 소켓 서버 전송

In [3]:
import mediapipe as mp
import cv2
import socket

# TCP 서버 설정
def send_pose_landmarks(host='127.0.0.1', port=6521):
    # 소켓 객체 생성
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
        # 서버에 연결
        client_socket.connect((host, port))
        print(f"서버에 연결되었습니다. {host}:{port}")

        # MediaPipe Pose 설정
        pose = mp.solutions.pose.Pose(
            static_image_mode=False,
            model_complexity=1,
            smooth_landmarks=True,
            enable_segmentation=False,  # 세그멘테이션 기능 비활성화
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5)

        # 그리기 함수 초기화
        mp_drawing = mp.solutions.drawing_utils

        # 웹캠 캡처
        cap = cv2.VideoCapture(0)

        while True:
            # 이미지 읽기
            ret, image = cap.read()
            if not ret:
                break

            # 이미지 변환
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # 포즈 추정
            results = pose.process(image)

            # 추정된 포즈 정보 확인
            if results.pose_landmarks:
                # 포즈 랜드마크를 리스트로 변환하여 문자열로 만듦
                pose_landmarks_str = ';'.join(
                    [f"{lm.x},{lm.y},{lm.z}" for lm in results.pose_landmarks.landmark])

                # 포즈 랜드마크 문자열 출력
                print(pose_landmarks_str)

                # 포즈 랜드마크 문자열을 서버로 전송
                client_socket.sendall(pose_landmarks_str.encode())

            # 이미지 출력
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp.solutions.pose.POSE_CONNECTIONS)
            cv2.imshow('MediaPipe Pose', image)

            # 키 입력 처리
            if cv2.waitKey(5) & 0xFF == ord('q'):
                break

        # 자원 해제
        cap.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    send_pose_landmarks()

서버에 연결되었습니다. 127.0.0.1:6521
